In [2]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
SPAM_PATH = os.path.join("datasets", "spam")

def fetch_spam_data(spam_url = SPAM_URL, spam_path = SPAM_PATH):
    if not os.path.isdir(spam_path):
        os.makedirs(spam_path)
        
    for filename, url in (("ham.tar.bz2", HAM_URL), ("spam.tar.bz2", SPAM_URL)):
        path = os.path.join(spam_path, filename)
        if not os.path.isfile(path):
            urllib.request.urlretrieve(url, path)
        tar_bz2_file = tarfile.open(path)
        tar_bz2_file.extractall(path=SPAM_PATH)
        tar_bz2_file.close()

In [3]:
fetch_spam_data()

In [7]:
HAM_DIR = os.path.join(SPAM_PATH, "easy_ham")
SPAM_DIR = os.path.join(SPAM_PATH, "spam")
ham_filenames = [name for name in sorted(os.listdir(HAM_DIR)) if len(name) > 20]
spam_filenames = [name for name in sorted(os.listdir(SPAM_DIR)) if len(name) > 20]

print(len(ham_filenames))
print(len(spam_filenames))


2500
500


In [10]:
import email
import email.policy

def load_email(is_spam, filename, spam_path=SPAM_PATH):
    directory = "spam" if is_spam else "easy_ham"
    with open(os.path.join(spam_path, directory, filename), "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [34]:
ham_emails = [load_email(is_spam=False, filename=name) for name in ham_filenames]
spam_emails = [load_email(is_spam=True, filename=name) for name in spam_filenames]

In [36]:
print(spam_emails[120].get_content().strip())

<html>

<head>
<meta http-equiv="Content-Language" content="en-us">
<meta http-equiv="Content-Type" content="text/html; charset=windows-1252">
<meta name="GENERATOR" content="Microsoft FrontPage 4.0">
<meta name="ProgId" content="FrontPage.Editor.Document">
<title>Does Your Computer Need an Oil Change</title>
</head>

<body>

<table border="0" width="538" height="1">
  <tr>
    <td width="538" height="1" align="center" bgcolor="#000000"><b><font face="Century Gothic" size="5" color="#FFFFFF">Does Your Computer Need an Oil
      Change?</font></b></td>
  </tr>
</table>
<table border="0" width="538" height="151">
  <tr>
    <td width="530" height="145"><b><font face="Tahoma" size="5">Norton</font><font color="#006600" face="Verdana" size="7"><br></font><i><font face="Verdana" color="#CC0000" size="7">SystemWorks
      2002</font></i><font size="4" face="Verdana"><br> </font><font face="Tahoma" size="5">Professional
      Edition</font> </b></td>
  </tr>
</table>
<table border="0" width="

In [26]:

def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return "multipart({})".format(", ".join([
            get_email_structure(sub_email)
            for sub_email in payload
        ]))
    else:
        return email.get_content_type()

In [40]:
for email in spam_emails:
    print(get_email_structure(email))

text/html
text/plain
text/plain
text/plain
text/plain
text/plain
text/plain
text/html
text/plain
text/html
text/plain
text/html
text/plain
text/html
text/html
text/plain
text/plain
text/plain
text/plain
text/plain
text/plain
multipart(text/plain, application/octet-stream)
multipart(text/html)
multipart(text/plain, text/html)
text/html
text/plain
text/plain
text/html
text/html
text/html
text/plain
text/plain
text/plain
text/plain
text/html
text/html
text/html
multipart(text/plain, text/html)
multipart(text/html)
text/plain
text/plain
text/html
text/plain
text/html
text/plain
text/html
text/plain
text/plain
text/html
text/html
text/plain
text/html
text/html
text/plain
text/html
text/plain
text/plain
text/html
text/html
text/plain
text/html
text/plain
text/plain
text/plain
text/plain
text/html
text/html
text/plain
text/plain
text/plain
text/plain
text/plain
text/plain
multipart(text/html)
text/plain
text/plain
text/plain
multipart(text/html)
text/plain
multipart(text/plain, text/html)
tex

In [63]:
from collections import Counter

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [64]:
structures_counter(ham_emails)

Counter({'text/plain': 2408,
         'multipart(text/plain, application/pgp-signature)': 66,
         'multipart(text/plain, text/html)': 8,
         'multipart(text/plain, text/enriched)': 1,
         'multipart(text/plain, application/ms-tnef, text/plain)': 1,
         'multipart(text/plain)': 3,
         'multipart(text/plain, application/octet-stream)': 2,
         'multipart(text/plain, text/plain)': 4,
         'multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)': 1,
         'multipart(text/plain, video/mng)': 1,
         'multipart(text/plain, multipart(text/plain))': 1,
         'multipart(text/plain, application/x-pkcs7-signature)': 1,
         'multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)': 1,
         'multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))': 1,
         'multipart(text/plain, application/x-java-applet)': 1})

In [65]:
structures_counter(spam_emails)

Counter({'text/html': 183,
         'text/plain': 218,
         'multipart(text/plain, application/octet-stream)': 1,
         'multipart(text/html)': 20,
         'multipart(text/plain, text/html)': 45,
         'multipart(text/plain)': 19,
         'multipart(text/html, text/plain)': 1,
         'multipart(text/html, application/octet-stream)': 2,
         'multipart(multipart(text/html))': 5,
         'multipart(text/plain, image/jpeg)': 3,
         'multipart(multipart(text/html), application/octet-stream, image/jpeg)': 1,
         'multipart(multipart(text/plain, text/html), image/gif)': 1,
         'multipart/alternative': 1})

In [69]:
spam_emails[23].items()

[('Return-Path', '<iq@insurancemail.net>'),
 ('Delivered-To', 'zzzz@localhost.spamassassin.taint.org'),
 ('Received',
  'from localhost (localhost [127.0.0.1])\tby phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 24BEC47CC8\tfor <zzzz@localhost>; Fri, 23 Aug 2002 06:12:01 -0400 (EDT)'),
 ('Received',
  'from phobos [127.0.0.1]\tby localhost with IMAP (fetchmail-5.9.0)\tfor zzzz@localhost (single-drop); Fri, 23 Aug 2002 11:12:01 +0100 (IST)'),
 ('Received',
  'from mail1.insuranceiq.com (host66.insuranceiq.com    [65.217.159.66] (may be forged)) by dogma.slashnull.org (8.11.6/8.11.6)    with ESMTP id g7MMxgZ28549 for <zzzz@jmason.org>; Thu, 22 Aug 2002 23:59:42    +0100'),
 ('Received',
  'from mail pickup service by mail1.insuranceiq.com with Microsoft    SMTPSVC; Thu, 22 Aug 2002 19:00:32 -0400'),
 ('Subject', 'Save 84% on CE Credits'),
 ('To', 'zzzz@spamassassin.taint.org'),
 ('Date', 'Thu, 22 Aug 2002 19:00:32 -0400'),
 ('From', 'IQ Campus <iq@insurancemail.net>'),
 ('Mess

In [70]:
spam_emails[23]["Subject"]

'Save 84% on CE Credits'

In [73]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    np.array(ham_emails + spam_emails), 
    np.array([0] * len(ham_emails) + [1] * len(spam_emails)),
    test_size = 0.2,
    random_state = 42
)

In [78]:
import re
from html import unescape

def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)

def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except: # in case of encoding issues
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content
    if html:
        return html_to_plain_text(html)


In [79]:
html_spam_emails = [email for email in X_train[y_train==1]
                    if get_email_structure(email) == "text/html"]
sample_html_spam = html_spam_emails[7]

In [82]:
sample_html_spam.get_content()[:1000]

'<HTML><HEAD><TITLE></TITLE><META http-equiv="Content-Type" content="text/html; charset=windows-1252"><STYLE>A:link {TEX-DECORATION: none}A:active {TEXT-DECORATION: none}A:visited {TEXT-DECORATION: none}A:hover {COLOR: #0033ff; TEXT-DECORATION: underline}</STYLE><META content="MSHTML 6.00.2713.1100" name="GENERATOR"></HEAD>\n<BODY text="#000000" vLink="#0033ff" link="#0033ff" bgColor="#CCCC99"><TABLE borderColor="#660000" cellSpacing="0" cellPadding="0" border="0" width="100%"><TR><TD bgColor="#CCCC99" valign="top" colspan="2" height="27">\n<font size="6" face="Arial, Helvetica, sans-serif" color="#660000">\n<b>OTC</b></font></TD></TR><TR><TD height="2" bgcolor="#6a694f">\n<font size="5" face="Times New Roman, Times, serif" color="#FFFFFF">\n<b>&nbsp;Newsletter</b></font></TD><TD height="2" bgcolor="#6a694f"><div align="right"><font color="#FFFFFF">\n<b>Discover Tomorrow\'s Winners&nbsp;</b></font></div></TD></TR><TR><TD height="25" colspan="2" bgcolor="#CCCC99"><table width="100%" bor

In [86]:
print(html_to_plain_text(sample_html_spam.get_content())[:1000])


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Watch for analyst "Strong Buy Recommendations" and several advisory newsletters picking CBYI.  CBYI has filed to be traded on the OTCBB, share prices historically INCREASE when companies get listed on this larger trading exchange. CBYI is trading around 25 cents and should skyrocket to $2.66 - $3.25 a share in the near future.
Put CBYI on your watch list, acquire a position TODAY.
REASONS TO INVEST IN CBYI
A profitable company and is on track to beat ALL earnings estimates!
One of the FASTEST growing distributors in environmental & safety equipment instruments.
Excellent management team, several EXCLUSIVE contracts.  IMPRESSIVE client list including the U.S. Air Force, Anheuser-Busch, Chevron Refining and Mitsubishi Heavy Industries, GE-Energy & Environmental Research.
RAPIDLY GROWING INDUSTRY
Industry revenues exceed $900 million, estimates indicate that there could be as much as $25 billi

In [88]:
print(email_to_text(spam_emails[23])[:100], "...")


 Save 84% on C.E. Credits & Pre-Licensing Courses
 
 State CE, CLU, CFP, CPA and Securities
 Unlimit ...


In [90]:
import nltk

stemmer = nltk.PorterStemmer()
for word in ["Computations", "Computation", "Computing", "Computed", "Compute", "Compulsive"]:
    print(word, "=>", stemmer.stem(word))

Computations => comput
Computation => comput
Computing => comput
Computed => comput
Compute => comput
Compulsive => compuls


In [92]:
try:
    import urlextract # may require an Internet connection to download root domain names
    
    url_extractor = urlextract.URLExtract()
    print(url_extractor.find_urls("Will it detect github.com and https://youtu.be/7Pq-S557XQU?t=3m32s"))
except ImportError:
    print("Error: replacing URLs requires the urlextract module.")
    url_extractor = None

['github.com', 'https://youtu.be/7Pq-S557XQU?t=3m32s']


In [93]:
from sklearn.base import BaseEstimator, TransformerMixin

class EmailToWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case=True, remove_punctuation=True,
                 replace_urls=True, replace_numbers=True, stemming=True):
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = []
        for email in X:
            text = email_to_text(email) or ""
            if self.lower_case:
                text = text.lower()
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url: len(url), reverse=True)
                for url in urls:
                    text = text.replace(url, " URL ")
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text, flags=re.M)
            word_counts = Counter(text.split())
            if self.stemming and stemmer is not None:
                stemmed_word_counts = Counter()
                for word, count in word_counts.items():
                    stemmed_word = stemmer.stem(word)
                    stemmed_word_counts[stemmed_word] += count
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [94]:
X_train[:3]

array([<email.message.EmailMessage object at 0x000002853E5B3908>,
      dtype=object)

In [129]:
X_few = X_train[:30]
X_few_wordcounts = EmailToWordCounterTransformer().fit_transform(X_few)
X_few_wordcounts

array([Counter({'chuck': 1, 'murcko': 1, 'wrote': 1, 'stuff': 1, 'yawn': 1, 'r': 1}),
       Counter({'the': 11, 'of': 9, 'and': 8, 'all': 3, 'christian': 3, 'to': 3, 'by': 3, 'jefferson': 2, 'i': 2, 'have': 2, 'superstit': 2, 'one': 2, 'on': 2, 'been': 2, 'ha': 2, 'half': 2, 'rogueri': 2, 'teach': 2, 'jesu': 2, 'some': 1, 'interest': 1, 'quot': 1, 'url': 1, 'thoma': 1, 'examin': 1, 'known': 1, 'word': 1, 'do': 1, 'not': 1, 'find': 1, 'in': 1, 'our': 1, 'particular': 1, 'redeem': 1, 'featur': 1, 'they': 1, 'are': 1, 'alik': 1, 'found': 1, 'fabl': 1, 'mytholog': 1, 'million': 1, 'innoc': 1, 'men': 1, 'women': 1, 'children': 1, 'sinc': 1, 'introduct': 1, 'burnt': 1, 'tortur': 1, 'fine': 1, 'imprison': 1, 'what': 1, 'effect': 1, 'thi': 1, 'coercion': 1, 'make': 1, 'world': 1, 'fool': 1, 'other': 1, 'hypocrit': 1, 'support': 1, 'error': 1, 'over': 1, 'earth': 1, 'six': 1, 'histor': 1, 'american': 1, 'john': 1, 'e': 1, 'remsburg': 1, 'letter': 1, 'william': 1, 'short': 1, 'again': 1, 'becom

In [182]:
[print(row, "=>", word_count) for row, (word_count) in enumerate(X_few_wordcounts)]

0 => Counter({'chuck': 1, 'murcko': 1, 'wrote': 1, 'stuff': 1, 'yawn': 1, 'r': 1})


TypeError: 'NoneType' object is not subscriptable

In [171]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                if len(word) > 1:
                    total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.most_common_ = most_common
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [172]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=10)
vocab_transformer.fit(X_few_wordcounts)

WordCounterToVectorTransformer(vocabulary_size=10)

In [173]:
vocab_transformer.most_common_

[('the', 141),
 ('to', 137),
 ('number', 113),
 ('and', 90),
 ('of', 81),
 ('it', 81),
 ('is', 57),
 ('url', 55),
 ('that', 55),
 ('thi', 52)]

In [174]:
vocab_transformer.vocabulary_

{'the': 1,
 'to': 2,
 'number': 3,
 'and': 4,
 'of': 5,
 'it': 6,
 'is': 7,
 'url': 8,
 'that': 9,
 'thi': 10}

In [175]:
vocab_transformer.transform(X_few_wordcounts).todense()

matrix([[  6,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [109,  11,   3,   0,   8,   9,   0,   0,   1,   1,   1],
        [ 66,   0,   3,   1,   2,   1,   0,   2,   4,   1,   1],
        [ 50,   1,   6,   0,   1,   1,   2,   1,   2,   0,   0],
        [100,   6,   2,   4,   2,   1,   1,   1,   1,   0,   0],
        [176,   5,   2,  11,   1,   3,   2,   4,   6,   2,   6],
        [181,   4,   7,   2,   4,   3,   2,   3,   3,   2,   1],
        [168,   5,   6,   0,   1,   0,   3,   1,   2,   4,   4],
        [ 40,   0,   0,   0,   0,   0,   0,   0,   2,   0,   0],
        [ 41,   1,   0,   8,   0,   0,   1,   0,   2,   0,   0],
        [128,   4,   7,   2,   0,   3,   1,   1,   0,   0,   2],
        [264,  16,   6,   6,   8,   5,   2,   3,   0,   8,   2],
        [105,   2,   4,   1,   3,   0,   2,   1,   0,   1,   0],
        [195,   9,   6,  29,   3,   1,   3,   4,   0,   5,   0],
        [678,   4,   7,  97,   2,  17,   6,   5,   3,   1,  18],
        [150,   4,   6,  

In [176]:
vocab_transformer.vocabulary_.get('url', 0)

8

In [177]:
from sklearn.pipeline import Pipeline

preprocess_pipeline = Pipeline([
    ("email_to_wordcount", EmailToWordCounterTransformer()),
    ("wordcount_to_vector", WordCounterToVectorTransformer())
])
X_train_transformed = preprocess_pipeline.fit_transform(X_train)

In [178]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_clf = LogisticRegression(solver="liblinear", random_state=42)
score = cross_val_score(log_clf, X_train_transformed, y_train, cv=5, verbose=3)
print(score)
print(score.mean())

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  ................................................................
[CV] .................................... , score=0.983, total=   0.1s
[CV]  ................................................................


C:\Users\ikuchin\AppData\Local\Continuum\miniconda3\envs\nnseries\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s


[CV] .................................... , score=0.990, total=   0.4s
[CV]  ................................................................
[CV] .................................... , score=0.985, total=   0.1s
[CV]  ................................................................
[CV] .................................... , score=0.990, total=   0.4s
[CV]  ................................................................
[CV] .................................... , score=0.994, total=   0.5s
[0.98333333 0.98958333 0.98541667 0.98958333 0.99375   ]
0.9883333333333335


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.4s finished


In [179]:
from sklearn.metrics import f1_score, recall_score, precision_score

X_test_transformed = preprocess_pipeline.transform(X_test)

log_clf.fit(X_train_transformed, y_train)
y_pred = log_clf.predict(X_test_transformed)


In [180]:
print("recall_score: {:.5f}%".format(recall_score(y_test, y_pred) * 100))
print("precision_score:  {:.5f}%".format(precision_score(y_test, y_pred) * 100))
print("f1:  {:.5f}%".format(f1_score(y_test, y_pred) * 100))



recall_score: 97.89474%
precision_score:  98.93617%
f1:  98.41270%
